In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [2]:
import base64
import pandas as pd
import re
import sys
import geopandas as gpd
import contextily as cx
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar

from ipywidgets import interact, interactive, interact_manual
import ipywidgets as widgets
from IPython.display import HTML, display

In [ ]:
sys.path.append("../..")
pd.options.display.max_rows = 500
pd.options.display.max_columns = 200
pd.options.display.max_colwidth = 100


display(HTML('''<style>
    .widget-label { min-width: 20ex !important; }
</style>'''))

from  sage_one_tree_planted.data.climate_and_economic_justice_dataset import ClimateAndEconomicJusticeDataset

c = ClimateAndEconomicJusticeDataset()
c.fetch_data()
[communities_csv] = c.screening_data.glob("communities-*.csv")
df2=pd.read_csv(communities_csv)
df_disadvantaged_communities = df2[df2["Identified as disadvantaged"] == True]
df_disadvantaged_communities.groupby(["County Name", "State/Territory"])["Identified as disadvantaged"]\
    .count().to_frame().sort_values("Identified as disadvantaged", ascending=False).head(100)

total_census_tracts_df = df2.groupby(["County Name", "State/Territory",])["Census tract ID"].count()
USA_LOCATIONS = gpd.read_file(c.shape_file/"usa.shp")


STATES=list(df2["State/Territory"].unique())

In [4]:
%matplotlib inline
def _is_disadvantaged_group(df):
    return df[df["Identified as disadvantaged"] == True]

def _compute_mean(df, columns):
    return df.groupby(["County Name", "State/Territory"])[columns].mean().reset_index()

def plot_on_usa_map(df_tmp: pd.DataFrame, locations=USA_LOCATIONS) -> None:
    df_tmp.rename(columns={"County Name":"CF", "State/Territory":"SF"}, inplace=True)
    usa_filtered_locations = locations[(locations["SF"].isin(set(df_tmp["SF"].tolist()))) & \
                                           (locations["CF"].isin(set(df_tmp["CF"].tolist())))]
    usa_filtered_locations_wm = usa_filtered_locations.to_crs(epsg=3857)
    fig, ax = plt.subplots(1, 1, figsize=(15,15))
    plt.close()
    usa_filtered_locations_wm.plot(ax=ax, figsize=(10, 10), alpha=0.5, edgecolor='k')
    cx.add_basemap(ax)
    ax.add_artist(ScaleBar(1))
    return fig

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

def _remove_boolen_filter(columns):
    res = []
    for column in columns:
        if not re.search("Greater.+", column):
            res.append(column)
    return res

# Identifying Disadvantaged Counties in the USA

## Methodology 

After starting to work with the data available on https://screeningtool.geoplatform.gov/en/methodology, we wanted to get a better idea of how which states and counties are most disadvantaged. 

We also wanted to give users the ability to explore those locations interactively based on user-set thresholds. 

Starting with the basics, we've created a tool that lets you identify which counties contain the most disadvantaged communities (Census Tracts) according to the disadvantaged critieria defined by the census. 


A census tract is identified as disadvantaged in one or more categories of criteria: 

IF the census tract is above the threshold for one or more environmental or climate indicators 

AND the census tract is above the threshold for the socioeconomic indicators 
(c.f. https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html) 




#### These results are for the USA as a whole. If you'd like to zoom in on an individual state, scroll down! 
##### Legend
- **topN** - the number of counties to show
- **min_census_tracts** a filter of the minimum number of disadvantaged census tracts that a county must contain in order to be displayed
- **Rank By**  By default, we rank each county by the percentage of disadvantaged census tracts that it contains. If you'd like to use the absolute value instead, just click
- **show_map** A toggle to display the counties on a map of the USA, N.B. this can be a bit slow. 

In [5]:
def show_disadvantaged_counties_usa(topN=10,  min_census_tracts=150, use_percent=True, show_map=False, ):
    if not use_percent:
        filtered = df_disadvantaged_communities.groupby(["County Name", "State/Territory"])["Identified as disadvantaged"]\
            .count().to_frame().join(total_census_tracts_df).rename(columns={"Census tract ID":"Total Census Tracts", "Identified as disadvantaged": "Total Census Tracts Identified as Disadvantaged"})\
            .sort_values("Total Census Tracts Identified as Disadvantaged", ascending=False)
        
        final_df = filtered[filtered["Total Census Tracts"] >= min_census_tracts ].reset_index().head(topN).reset_index()[["County Name", "State/Territory","Total Census Tracts", "Total Census Tracts Identified as Disadvantaged" ]]
        if show_map:
            display(plot_on_usa_map(final_df))
        display(final_df)
        return final_df
    else: 
        with_totals_df = df2.groupby(["County Name", "State/Territory", "Identified as disadvantaged"])[["Identified as disadvantaged"]].count()
        with_totals_df.columns = ["count"]
        with_total_df = with_totals_df.join(total_census_tracts_df)
        with_total_df["Percent of Census Tracts Identified as Disadvantaged"] = 100 * with_total_df["count"] / with_total_df["Census tract ID"]
        with_total_df.reset_index(inplace=True)
        with_total_df.rename(columns={"Census tract ID":"Total Census Tracts"}, inplace=True)
        filtered  = with_total_df[with_total_df["Identified as disadvantaged"]  == True]\
.sort_values("Percent of Census Tracts Identified as Disadvantaged", ascending=False).reset_index()
        
        final_df = filtered[filtered["Total Census Tracts"] >= min_census_tracts ].head(topN).reset_index()[["County Name", "State/Territory","Total Census Tracts", "Percent of Census Tracts Identified as Disadvantaged" ]]
        if show_map:
            display(plot_on_usa_map(final_df))
        display(final_df)
        return final_df
w = interactive(show_disadvantaged_counties_usa, topN=(10,500,10), min_census_tracts=(1, 1000, 5), use_percent = widgets.Dropdown(
    options=[("Percentage of tracts", True), ("Total tracts", False)],
    description='Rank by',
)
         )

output = widgets.Output()
def generate_download_link_usa(arg):
    output.clear_output()
    with output:
        display(create_download_link(w.result, filename="Ranked Disadvantaged Counties"))
    return

for child in w.children:
    child.observe(generate_download_link_usa)
display(output, w)


Output()

interactive(children=(IntSlider(value=10, description='topN', max=500, min=10, step=10), IntSlider(value=150, …

# Identifying Disadvantaged Counties per state

The methodology here is the same as the above, but now you can zoom in and identify the m


In [10]:
def show_disadvantaged_counties_state(state=STATES, topN=10,  min_census_tracts=150, use_percent=True, show_map=False, ):
    if not use_percent:
        filtered = df_disadvantaged_communities[df_disadvantaged_communities["State/Territory"]==state].groupby(["County Name", "State/Territory"])["Identified as disadvantaged"]\
            .count().to_frame().join(total_census_tracts_df).rename(columns={"Census tract ID":"Total Census Tracts", "Identified as disadvantaged": "Total Census Tracts Identified as Disadvantaged"})\
            .sort_values("Total Census Tracts Identified as Disadvantaged", ascending=False)
        
        final_df = filtered[filtered["Total Census Tracts"] >= min_census_tracts ].reset_index().head(topN).reset_index()[["County Name", "State/Territory","Total Census Tracts", "Total Census Tracts Identified as Disadvantaged" ]]
        if show_map:
            display(plot_on_usa_map(final_df))
        display(final_df)
        return final_df
    else: 
        with_totals_df = df2[df2["State/Territory"] == state].groupby(["County Name", "State/Territory", "Identified as disadvantaged"])[["Identified as disadvantaged"]].count()
        with_totals_df.columns = ["count"]
        with_total_df = with_totals_df.join(total_census_tracts_df)
        with_total_df["Percent of Census Tracts Identified as Disadvantaged"] = 100 * with_total_df["count"] / with_total_df["Census tract ID"]
        with_total_df.reset_index(inplace=True)
        with_total_df.rename(columns={"Census tract ID":"Total Census Tracts"}, inplace=True)
        filtered  = with_total_df[with_total_df["Identified as disadvantaged"]  == True]\
.sort_values("Percent of Census Tracts Identified as Disadvantaged", ascending=False).reset_index()
        
        final_df = filtered[filtered["Total Census Tracts"] >= min_census_tracts ].head(topN).reset_index()[["County Name", "State/Territory","Total Census Tracts", "Percent of Census Tracts Identified as Disadvantaged" ]]
        if show_map:
            display(plot_on_usa_map(final_df))
        display(final_df)
        return final_df

w_state = interactive(show_disadvantaged_counties_state, topN=(10,500,10), min_census_tracts=(1, 1000, 5), use_percent = widgets.Dropdown(
    options=[("Percentage of tracts", True), ("Total tracts", False)],
    description='Rank by',
)
         )
output_state = widgets.Output()
def generate_download_link_state(arg):
    output_state.clear_output()
    with output_state:
        display(create_download_link(w_state.result, filename=f"Ranked Disadvantaged Counties in {w_state.kwargs.get('state', 'your state')}"))
    return

for child in w_state.children:
    child.observe(generate_download_link_state)

display(output_state, w_state)

Output()

interactive(children=(Dropdown(description='state', options=('Alabama', 'Alaska', 'Arizona', 'Arkansas', 'Cali…

# Identifying Disadvantaged Counties by Specific Criteria

The workbook below provides you with the ability to find the most disadvantaged counties by a criteria of your choice. 

##### Legend
- **criteria** - the criteria for disadvantaged census tracts that you would like to use.

In [21]:
COLUMNS_FILTER = df2.columns.tolist()[3:]
COLUMNS_FILTER.remove("Identified as disadvantaged")
COLUMNS_FILTER = _remove_boolen_filter(COLUMNS_FILTER)
df_mean =_compute_mean(df=df_disadvantaged_communities , columns=COLUMNS_FILTER)

In [22]:
pd.options.mode.chained_assignment = None

def _rank_by_column(criteria=COLUMNS_FILTER, topN=10, state=STATES, show_map=False):
    df_tmp = df_mean[df_mean["State/Territory"] == state].copy()
    df_tmp["max_rank"] = df_tmp.groupby("State/Territory")[criteria].rank(method="max")
    df_final = df_tmp[df_tmp["max_rank"] <= topN]
    if show_map:
        display(plot_on_usa_map(df_final, locations=USA_LOCATIONS))
    display(df_final)
    return df_final
by_criteria_w = interactive(_rank_by_column)

output_criteria = widgets.Output()
def generate_download_link_criteria(arg):
    output_criteria.clear_output()
    with output_criteria:
        display(create_download_link(by_criteria_w.result, filename=f"Counties in {by_criteria_w.kwargs.get('state', 'your state')} ranked by {by_criteria_w.kwargs.get('column', 'default')}"))
    return

for child in by_criteria_w.children:
    child.observe(generate_download_link_criteria)

display(output_criteria, by_criteria_w)

Output()

interactive(children=(Dropdown(description='criteria', options=('Total threshold criteria exceeded', 'Total ca…

# Identifying disadvantaged states by one or more chosen criteria

In [ ]:
# Can use percentile columns as options for ranking
percentile_cols = [x for x in df2.columns if "(percentile)" in x]
state_col = "State/Territory"

In [ ]:
def plot_states(df_tmp: pd.DataFrame, locations=USA_LOCATIONS) -> None:
    states = set(df_tmp.index.values)
    usa_filtered_locations = locations[(locations["SF"].isin(states))]
    usa_filtered_locations_wm = usa_filtered_locations.to_crs(epsg=3857)
    fig, ax = plt.subplots(1, 1, figsize=(15,15))
    plt.close()
    usa_filtered_locations_wm.plot(ax=ax, figsize=(10, 10), alpha=0.5, edgecolor='k')
    cx.add_basemap(ax)
    ax.add_artist(ScaleBar(1))
    return fig

In [ ]:
layout = {'width': 'max-content', 'height': '150px'}
style = {'description_width': '150px'}
def _state_ranking_by_criteria(criteria_cols=[], topN=10, show_map=False):
    cols = list(criteria_cols)  + [state_col]
    reduced_df = df2.loc[:,cols]
    agg_df = reduced_df.groupby([state_col]).mean()
    agg_df.dropna(inplace=True)
    agg_df.columns = [f"Mean {col} across state" for col in agg_df.columns]
    agg_df['Mean Percentile of Chosen Criteria'] = agg_df.mean(axis=1)
    agg_df.sort_values('Mean Percentile of Chosen Criteria', inplace=True, ascending=False)
    agg_df = agg_df.round(decimals=2)
    topN_df = agg_df.head(topN)
    if show_map:
        display(plot_states(topN_df, locations=USA_LOCATIONS))
    display(topN_df)
    return topN_df

criteria_selector = widgets.SelectMultiple(
    layout = layout,
    style = style,
    options = percentile_cols,
    description = "Chosen ranking criteria:",
    disabled = False
)
state_by_criteria = interactive(
    _state_ranking_by_criteria, 
    criteria_cols = criteria_selector,
    topN = (1, len(STATES), 1),
    show_map = False
)
output_criteria = widgets.Output()
def generate_download_link_criteria(arg):
    output_criteria.clear_output()
    with output_criteria:
        # Data downloaded should contain columns of chosen criteria only
        display(create_download_link(state_by_criteria.result, filename="States ranked by custom criteria"))
    return
for child in state_by_criteria.children:
    child.observe(generate_download_link_criteria)
display(state_by_criteria)